In [1]:
import numpy as np
from MyGenerator import DataGenerator
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
import pickle
import os

Using TensorFlow backend.


In [2]:
with open(os.path.join("regression_data", 'labels.pickle'), 'rb') as handle:
    labels = pickle.load(handle)

In [3]:
list_of_labels = [*labels]
np.random.shuffle(list_of_labels)

In [4]:
len(labels)

50000

In [5]:
partition = {'train': list_of_labels[:40000], 
             'validation': list_of_labels[40000:45000],
             'test': list_of_labels[45000:50000]}
y_label = {'train': [labels[k] for k in partition['train']], 
           'validation': [labels[k] for k in partition['validation']],
           'test': [labels[k] for k in partition['test']]}

In [6]:
len(y_label['validation'])

5000

In [7]:
len(partition['validation'])

5000

In [8]:
params = {'dim': (2500,),
          'batch_size': 32,
          'n_channels': 1,
          'shuffle': True}
test_params = {'dim': (2500,),
               'batch_size': len(partition['test']),
               'n_channels': 1,
               'shuffle': False}

In [9]:
training_generator = DataGenerator(partition['train'], y_label['train'],  **params)
validation_generator = DataGenerator(partition['validation'], y_label['validation'],  **params)
testing_generator = DataGenerator(partition['test'], y_label['test'],  **test_params)

In [10]:
inputs = keras.Input(shape = (2500,1))
conv_1_1 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(inputs)
conv_1_2 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_1)
conv_1_3 = keras.layers.Conv1D(32, kernel_size = 3, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_1_2)
pool_1 = keras.layers.MaxPool1D(pool_size = 4)(conv_1_3)

conv_2_1 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_1)
conv_2_2 = keras.layers.Conv1D(64, kernel_size = 5, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(conv_2_1)
#conv_2_3 = keras.layers.Conv1D(64, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_2_2)
pool_2 = keras.layers.MaxPool1D(pool_size = 4)(conv_2_2)

conv_3_1 = keras.layers.Conv1D(128, kernel_size = 7, activation = 'relu', 
                            kernel_initializer = keras.initializers.he_normal())(pool_2)
#conv_3_2 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_1)
#conv_3_3 = keras.layers.Conv1D(128, kernel_size = 3, activation = 'relu', 
#                            kernel_initializer = keras.initializers.he_normal())(conv_3_2)
pool_3 = keras.layers.MaxPool1D(pool_size = 4)(conv_3_1)



flat_1 = keras.layers.Flatten()(pool_3)



dense_1 = keras.layers.Dense(1024, activation = 'relu', 
                             kernel_initializer = keras.initializers.he_normal())(flat_1)
outputs = keras.layers.Dense(2, kernel_initializer = keras.initializers.he_normal())(dense_1)
model = keras.Model(inputs = inputs, outputs = outputs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2500, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2498, 32)          128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2496, 32)          3104      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2494, 32)          3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 623, 32)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 619, 64)           10304     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 615, 64)           20544     
__________

In [11]:
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.lin_reg.hdf5', 
                               verbose=2, save_best_only=True)
model.compile(optimizer=adam,
              loss='mean_squared_error', metrics = ['mae'])
history = model.fit_generator(generator=training_generator,
                              validation_data=validation_generator,
                              #steps_per_epoch=training_generator.__len__(),
                              #workers=2,
                              callbacks=[checkpointer],
                              epochs = 50,
                              use_multiprocessing=True)

Epoch 1/50
1250/1250 [==============================] - 23s 18ms/step - loss: 1.0177 - mean_absolute_error: 0.4211 - val_loss: 0.2766 - val_mean_absolute_error: 0.2372

Epoch 00001: val_loss improved from inf to 0.27659, saving model to saved_models/weights.best.lin_reg.hdf5
Epoch 2/50
1250/1250 [==============================] - 17s 14ms/step - loss: 0.2336 - mean_absolute_error: 0.2031 - val_loss: 0.2942 - val_mean_absolute_error: 0.3247

Epoch 00002: val_loss did not improve from 0.27659
Epoch 3/50
1250/1250 [==============================] - 18s 14ms/step - loss: 0.1861 - mean_absolute_error: 0.1740 - val_loss: 0.1555 - val_mean_absolute_error: 0.1555

Epoch 00003: val_loss improved from 0.27659 to 0.15550, saving model to saved_models/weights.best.lin_reg.hdf5
Epoch 4/50
1250/1250 [==============================] - 18s 14ms/step - loss: 0.1616 - mean_absolute_error: 0.1602 - val_loss: 0.1405 - val_mean_absolute_error: 0.1488

Epoch 00004: val_loss improved from 0.15550 to 0.14052,

1250/1250 [==============================] - 18s 14ms/step - loss: 0.0733 - mean_absolute_error: 0.0982 - val_loss: 0.0672 - val_mean_absolute_error: 0.1011

Epoch 00031: val_loss did not improve from 0.06666
Epoch 32/50
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0726 - mean_absolute_error: 0.0982 - val_loss: 0.0667 - val_mean_absolute_error: 0.1001

Epoch 00032: val_loss did not improve from 0.06666
Epoch 33/50
1250/1250 [==============================] - 18s 14ms/step - loss: 0.0723 - mean_absolute_error: 0.0977 - val_loss: 0.0666 - val_mean_absolute_error: 0.1014

Epoch 00033: val_loss improved from 0.06666 to 0.06663, saving model to saved_models/weights.best.lin_reg.hdf5
Epoch 34/50
1250/1250 [==============================] - 17s 14ms/step - loss: 0.0704 - mean_absolute_error: 0.0969 - val_loss: 0.0678 - val_mean_absolute_error: 0.1028

Epoch 00034: val_loss did not improve from 0.06663
Epoch 35/50
1250/1250 [==============================] - 17s 14ms/st

In [12]:
model.load_weights('saved_models/weights.best.lin_reg.hdf5')
model.evaluate_generator(testing_generator)

[0.09784813225269318, 0.099308080971241]

In [13]:
partition['test'][4]

'linear_7546'

In [14]:
y_label['test'][4]

[8.62896652601637, 1.6399030358144628]

In [17]:
k = 54
X_test = np.zeros((1,2500,1))
X_test[0,] = np.load("regression_data/" + partition['test'][k] + '.npy')[:,np.newaxis]

In [18]:
print(model.predict(X_test), y_label['test'][k] )

[[3.3109837 1.358473 ]] [3.336433998445978, 1.3866733965257403]


In [19]:
model.save('saved_models/linear_model.h5')